In [3]:
import pandas as pd

from binance.client import Client
from model import OnnxModel
from pathlib import Path

model = OnnxModel(
    model_path=Path("models/btc_usdt/model.onnx"), 
    scaler_path=Path("models/btc_usdt/scaler.pkl")
)
binance_client = Client(
    api_key="replace", 
    api_secret="replace"
)
klines = binance_client.get_historical_klines("BTCUSDT", "1h", "5 days ago UTC")
data = pd.DataFrame(
    klines, 
    columns = [
        "timestamp", 
        "open", 
        "high", 
        "low", 
        "close", 
        "volume", 
        "close_time", 
        "quote_av", 
        "trades", 
        "tb_base_av", 
        "tb_quote_av", 
        "ignore"
    ],
)
data["timestamp"] = pd.to_datetime(data["timestamp"], unit="ms")

In [12]:
type(data["close"][0])

str

In [13]:
rows = []
for _, row in data.iterrows():
    row_data = dict(
        day_of_week=row["timestamp"].dayofweek,
        day_of_month=row["timestamp"].day,
        week_of_year=row["timestamp"].week,
        month_of_year=row["timestamp"].month,
        open=row["open"],
        high=row["high"],
        low=row["low"],
        close=row["close"],
        close_change=float(row["close"]) - float(row["open"]),
    )
    rows.append(row_data)
df = pd.DataFrame(rows)

In [15]:
scaled_data = pd.DataFrame(
    model.scaler.transform(df),
    index=df.index,
    columns=df.columns,
)
scaled_data.head()

,day_of_week,day_of_month,week_of_year,month_of_year,open,high,low,close,close_change
0,-0.666667,-1.0,-1.0,-0.818182,-0.511170,-0.532073,-0.512932,-0.528152,-0.059989
1,-0.666667,-1.0,-1.0,-0.818182,-0.528170,-0.547567,-0.518942,-0.536024,-0.011066
2,-0.666667,-1.0,-1.0,-0.818182,-0.536041,-0.540439,-0.517532,-0.529221,0.067574
3,-0.666667,-1.0,-1.0,-0.818182,-0.529199,-0.536196,-0.511428,-0.515339,0.105309
4,-0.666667,-1.0,-1.0,-0.818182,-0.515357,-0.534453,-0.509287,-0.519805,0.007186


In [63]:
import torch

def create_single_sequence(input_data: pd.DataFrame, target_column: str):
    return torch.Tensor(input_data.values).unsqueeze(0)


In [64]:
X = create_single_sequence(scaled_data, "close")
X.shape

torch.Size([1, 120, 9])

In [65]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [66]:
inputs = {model.model.get_inputs()[0].name: to_numpy(X)}
outputs = model.model.run(None, inputs)

In [77]:
outputs[0][0]

array([-0.42638165], dtype=float32)